Komputerówka - wersja 1. Do zdobycia - 33p.

Proszę nie tracić czasu na fail-safe'y dla parametrów funkcji, kod ma po prostu działać.

Jeśli kod skompiluje się wam w trakcie komputerówki to jego optymalnośc jest wystarczająca.

Proszę czytać ze zrozumieniem i jeśli to podane używać konkretnych metod. Kod powinien być czysty i bez zbędnych elementów.

Zabronione jest korzystanie z chatbotów, urządzeń elektronicznych poza pojedynczym komputerem, komunikatorów wszelkiego typu (automatyczne 0 punktów bez możliwości poprawy).

Przykładowe rozwiązanie:
Zadanie X - napisz funkcję FUN($a,b$) liczącą sumę dwóch podanych liczb. Jako przykład użyj $a=1,b=2$. Prawidłowa odpowiedź to $3$.

In [1]:
def FUN(a,b):
    return a+b
a, b = 1, 2
print('wynik to ' + str(FUN(a,b)))

wynik to 3


Zadanie 1 [8p] - Niech $X$ będzie zmienną losową typu ciągłego o dodatnim nośniku i gęstości $f(x)=(4x)^{-1/2}e^{-\sqrt{x}}$ (uogólniony rozkład Gamma). Napisz funkcję GEN($N$) przyjmującą liczbę naturalną $N$ i zwracającą wektor długości $N$ zawierający próbę IID zmiennych $X_i\stackrel{D}{=}X$. Użyj metody odwrotnej dystrybuanty. Za jej pomocą wygeneruj próbę wielkości $N=10^7$, którą wykorzystaj do stworzenia histogramu porównanego z gęstością teoretyczną $f(x)$. Zadbaj o odpowiednie opisy na wykresie.

In [5]:
import numpy as np
import scipy

Zadanie 2 [11p] - Niech $W^1_t$, $W^2_t$ będą dwoma niezależnymi ruchami Browna (procesami Wienera). Napisz funkcję FUN($R$, $\Delta t$, N) przyjmującą krok dyskretyzacji czasu $\Delta t$, $R>0$ oraz próbę Monte-Carlo $N$, która zwróci wyestymowaną średnią zmiennej $T$ będącej czasem pierwszego wyjścia procesu dwuwymiarowego procesu $(W^1_t,W^2_t)$ poza koło $K(R,0)$ o promieniu $R$ i środku $0$, czyli
$$ T=\inf_{t\geq 0}\{t:(W^1_t,W^2_t)\in K(R,0)^C\}.$$ Jako przykładu użyj parametrów $R=\frac{1}{2}$, $\Delta t=10^{-4}$, $N=10^4$. Poprawny wynik to $\frac{1}{2}R^2=\frac{1}{8}$

In [2]:
def FUN(R, dt, N):
    times = np.zeros(N)
    for i in range(N):
        x = 0
        y = 0
        t = 0
        while x**2 + y**2 <= R**2:
            x += np.random.normal(0, 1) * np.sqrt(dt)
            y += np.random.normal(0, 1) * np.sqrt(dt)
            t += dt
        times[i] = t
    return np.sum(times) / N

In [4]:
print(FUN(1/2, 10**(-4), 10000))

0.12841203999999784


Zadanie 3 [14p] - Napisz funkcję ESTRISK($u,c,\lambda,\mu,T,M$) estymującą prawdopodobieństwo ruiny klasycznej w czasie $T$ procesu
$$ X_t=u+ct-\sum_{i=0}^{N_t}Y_i, $$
gdzie $N_t$ to niejednorodny proces Poissona o intensywnosci $\lambda(t)$, $Y_i\sim Exp(\mu)$, $E(Y_i)=\mu$, $u,c>0$ a $M$ to wielkośc próby Monte-Carlo. Wyestymuj prawdopodobieństwo ruiny klasycznej dla parametrów $u=5$,$c=3$, $T=1$, $\lambda(t)=\sin^2(\pi t)+\ln(1+3t)$, $\mu=5$, $M=10^4$. Do generowania $N_t$ możesz użyć dowolnej metody poznanej na zajęciach. Poprawna odpowiedź to około $0.365$.

In [27]:
from scipy.interpolate import interp1d
u = 5
c = 3
T = 1
lambda_func = lambda t: np.sin(np.pi*t)**2 + np.log(1 + 3*t)
mi = 5
n = 10**4 #liczba punktów do szacowania funkcji odwrotnej

def niejednorodny_proces_poissona(lambda_func, T):
    lambda2 = scipy.integrate.quad(lambda_func, 0, T)[0]
    k = np.random.poisson(lambda2)
    xs = np.linspace(0, T, 1000)
    M = max(lambda_func(xs))

    skoki = np.zeros(k)
    i = 0
    while i < k:
        ux = np.random.uniform(0, T)
        uy = np.random.uniform(0, M)
        if uy < lambda_func(ux):
            skoki[i] = ux
            i += 1
            continue
        else:
            continue

    skoki.sort()
    return k, skoki

In [42]:

def simulate_poisson_nonhomogeneous(lambda_func, T):
    # Metoda odrzutowa (thinning method) do generowania niejednorodnego procesu Poissona
    max_lambda = max(lambda_func(t) for t in np.linspace(0, T, 1000))
    times = np.zeros(n)
    t = 0
    i = 0
    while t < T:
        t += np.random.exponential(1 / max_lambda)
        if t < T and np.random.uniform(0, 1) < lambda_func(t) / max_lambda:
            times[i] = t
            i += 1
    return len(times),times

In [46]:
def proces_ruiny(eta, u, c, T, lambda_func):
    num_events, event_times = niejednorodny_proces_poissona(lambda_func, T)
    etas = np.random.exponential(eta, num_events)
    etas_cumsum = np.cumsum(etas)
    R = u + c*event_times - etas_cumsum
    try:
        if np.min(R) < 0:
            return 1
        else:
            return 0
    except:
        pass

In [47]:
def ruin_prob_estim(eta, u, c, T, lambda_func, mc):
    if_ruin = np.zeros(mc)
    for i in range(mc):
        if_ruin[i] = proces_ruiny(eta, u, c, T, lambda_func)
    return sum(if_ruin)/len(if_ruin)

In [71]:
print(niejednorodny_proces_poissona(lambda_func, T))

(1, array([0.83668368]))


In [77]:
print(simulate_poisson_nonhomogeneous(lambda_func, T))

(10000, array([0.59930625, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]))


In [80]:
print(ruin_prob_estim(mi, u, c, T, lambda_func, 1000))

nan
